In [1]:
# Running different evaluations for long-doc-summ

"""
to implement:
R-1, R-2, R-L, BERTScore, METEOR
Automatic metrics to compare against previous papers
SNaC for coherence
Full code in this repo https://github.com/tagoyal/snac
Should be easy to run their T5 models unless I’m retraining
QAFactEval for faithfulness
Full code in this repo https://github.com/salesforce/QAFactEval#using-qafacteval
Should be easy to run but not tested for my domain
Need automatic metric for coverage
AI-eval
Just ask GPT-4 the same questions as to the humans
See what Faisal is doing with this
https://arxiv.org/pdf/2305.17926.pdf
Human-eval
Is the summary correct relative to the passage?
Does the summary cover the important events?
Is the summary well-written and coherent?
Preference relative to baseline summary
Writing class will evaluate summaries of their own work
Add interpretation here
What do we want to get out of the eval? Have this drive the approach
Strategies from here https://arxiv.org/pdf/2301.13298.pdf
Error analysis on low quality summaries
"""

'\nto implement:\nR-1, R-2, R-L, BERTScore, METEOR\nAutomatic metrics to compare against previous papers\nSNaC for coherence\nFull code in this repo https://github.com/tagoyal/snac\nShould be easy to run their T5 models unless I’m retraining\nQAFactEval for faithfulness\nFull code in this repo https://github.com/salesforce/QAFactEval#using-qafacteval\nShould be easy to run but not tested for my domain\nNeed automatic metric for coverage\nAI-eval\nJust ask GPT-4 the same questions as to the humans\nSee what Faisal is doing with this\nhttps://arxiv.org/pdf/2305.17926.pdf\nHuman-eval\nIs the summary correct relative to the passage?\nDoes the summary cover the important events?\nIs the summary well-written and coherent?\nPreference relative to baseline summary\nWriting class will evaluate summaries of their own work\nAdd interpretation here\nWhat do we want to get out of the eval? Have this drive the approach\nStrategies from here https://arxiv.org/pdf/2301.13298.pdf\nError analysis on low

In [2]:
orig_path = "booksumm-sample2/what-maisie-knew"

In [3]:
import json

with open(orig_path+"/metadata.json") as f:
  book_metadata = json.load(f)

In [4]:
book_metadata

{'title': 'What Maisie Knew',
 'author': 'Henry James',
 'total-chapters': 31,
 'average-chapter-len': 4256.935483870968,
 'dir': '7118-chapters'}

In [5]:
import os
import json
import pandas as pd

# Define directory paths
dir_path1 = "booksumm-sample2/what-maisie-knew"
dir_path2 = "gpt4_summaries/what-maisie-knew"

# Define the data structure
data = {
    "subdir": [],
    "metadata": [],
    "content": [],
    "summaries": [],
    "new_summary": []
}

# Loop over subdirectories
for subdir in os.listdir(dir_path1):
    subdir_path1 = os.path.join(dir_path1, subdir)

    # If the path is a directory, process it
    if os.path.isdir(subdir_path1):
        data["subdir"].append(subdir)

        # Process metadata.json
        with open(os.path.join(subdir_path1, "metadata.json"), 'r') as f:
            data["metadata"].append(json.load(f))

        # Process content.txt
        with open(os.path.join(subdir_path1, "content.txt"), 'r') as f:
            data["content"].append(f.read())

        # Process summaries
        summaries = []
        for file in os.listdir(subdir_path1):
            if file.endswith("summary.txt"):
                with open(os.path.join(subdir_path1, file), 'r') as f:
                    content = f.read()
                    summaries.append(content)
        data["summaries"].append(summaries)

        # Process new directory
        subdir_path2 = os.path.join(dir_path2, subdir)
        if os.path.isdir(subdir_path2):
            with open(os.path.join(subdir_path2, "summary.txt"), 'r') as f:
                data["new_summary"].append(f.read())
        else:
            data["new_summary"].append(None)

# Create a DataFrame
df = pd.DataFrame(data)
df = df.set_index('subdir')  # Set index to subdirectory names


json.loads(df['summaries'][0][0])["summary"]


'Maisie has to go back to her father and Miss Overmore . This means that she has to say goodbye to Mrs. Wix for the first time, and this is super, super sad! Maisie asks her father if he liked Miss Overmore "just the same" while she was at her mother\'s . This embarrasses Miss Overmore, even though Beale Farange makes no secret of having been with Miss Overmore the whole time. This is just above Maisie\'s head. She, too, feels embarrassed and wonders why Miss Overmore has been awkward. She also tries to figure out the nature of her father\'s relationship to Miss Overmore. Maisie takes to treating her doll, Lisette, the way she herself is treated by her mother and Miss Overmore. She keeps secrets from her doll and even reprimands her for asking indiscreet questions.'

In [6]:
df

,metadata,content,summaries,new_summary
subdir,,,,
05,"{'book-id': '7118-chapters', 'chapter-id': '05...",\n\nThe second parting from Miss Overmore had ...,"[{""name"": ""Chapter V"", ""url"": ""https://web.arc...",This chapter revolves around a young girl name...
21,"{'book-id': '7118-chapters', 'chapter-id': '21...",\n\nA good deal of the rest of Ida's visit was...,"[{""name"": ""Chapter XXI"", ""url"": ""https://web.a...","In this chapter, Maisie and her mother, Ida, s..."
30,"{'book-id': '7118-chapters', 'chapter-id': '30...",\n\nAfter they were seated there it was differ...,"[{""name"": ""Chapter XXX"", ""url"": ""https://web.a...","In this chapter, Sir Claude and Maisie have a ..."
23,"{'book-id': '7118-chapters', 'chapter-id': '23...",\n\nSir Claude was stationed at the window; he...,"[{""name"": ""Chapter XXIII"", ""url"": ""https://web...","In this chapter, Mrs. Wix, Sir Claude, and Mai..."
14,"{'book-id': '7118-chapters', 'chapter-id': '14...",\n\nMrs Beale fairly swooped upon her and the ...,"[{""name"": ""Chapter XIV"", ""url"": ""https://web.a...","In this chapter, Maisie is reunited with her s..."


In [7]:
def process_summary(summary_list):
  new_list = []
  for summary in summary_list:
    out = json.loads(summary)["summary"]
    new_list.append(out)
  return new_list


In [8]:
df["summaries"] = df['summaries'].apply(process_summary)

In [9]:
df

,metadata,content,summaries,new_summary
subdir,,,,
05,"{'book-id': '7118-chapters', 'chapter-id': '05...",\n\nThe second parting from Miss Overmore had ...,[Maisie has to go back to her father and Miss ...,This chapter revolves around a young girl name...
21,"{'book-id': '7118-chapters', 'chapter-id': '21...",\n\nA good deal of the rest of Ida's visit was...,[Ida and Maisie have a heart-to-heart. Ida is ...,"In this chapter, Maisie and her mother, Ida, s..."
30,"{'book-id': '7118-chapters', 'chapter-id': '30...",\n\nAfter they were seated there it was differ...,[Maisie's cafe trip with Sir Claude turns into...,"In this chapter, Sir Claude and Maisie have a ..."
23,"{'book-id': '7118-chapters', 'chapter-id': '23...",\n\nSir Claude was stationed at the window; he...,[Mrs. Wix says that she has seen Maisie's moth...,"In this chapter, Mrs. Wix, Sir Claude, and Mai..."
14,"{'book-id': '7118-chapters', 'chapter-id': '14...",\n\nMrs Beale fairly swooped upon her and the ...,[Mrs. Beale lets it slip that she and Sir Clau...,"In this chapter, Maisie is reunited with her s..."


In [10]:
%pip install rouge_score
%pip install nltk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import evaluate
meteor = evaluate.load("meteor")
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")

/Users/mbondarenko/opt/anaconda3/envs/long-doc-summ/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mbondarenko/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mbondarenko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/mbondarenko/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [12]:

predictions = [df["new_summary"][0]]
references = [df["summaries"][0]]
print(rouge.compute(predictions=predictions,references=references))
print(meteor.compute(predictions=predictions,references=references))
print(bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased"))

INFO:absl:Using default tokenizer.


{'rouge1': 0.41354723707664887, 'rouge2': 0.11091234347048301, 'rougeL': 0.19576719576719573, 'rougeLsum': 0.25396825396825395}
{'meteor': 0.35806239548866164}
{'precision': [0.7972561120986938], 'recall': [0.8193682432174683], 'f1': [0.7973999381065369], 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=3.0.2)'}


In [13]:
from evaluate import load
bertscore = load("bertscore")
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
print(results)

{'precision': [0.7972561120986938], 'recall': [0.8193682432174683], 'f1': [0.7973999381065369], 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=3.0.2)'}


In [4]:
from qafacteval import QAFactEval
kwargs = {"cuda_device": 0, "use_lerc_quip": True, \
        "verbose": True, "generation_batch_size": 32, \
        "answering_batch_size": 32, "lerc_batch_size": 8}

model_folder = "" # path to models downloaded with download_models.sh
metric = QAFactEval(
    lerc_quip_path=f"{model_folder}/quip-512-mocha",
    generation_model_path=f"{model_folder}/generation/model.tar.gz",
    answering_model_dir=f"{model_folder}/answering",
    lerc_model_path=f"{model_folder}/lerc/model.tar.gz",
    lerc_pretrained_model_path=f"{model_folder}/lerc/pretraining.tar.gz",
    **kwargs
)

results = metric.score_batch_qafacteval(["This is a source document"], [["This is a summary."]], return_qa_pairs=True)
score = results[0][0]['qa-eval']['lerc_quip']

/Users/mbondarenko/opt/anaconda3/envs/long-doc-summ/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
DEPRECATION: https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.5/en_core_web_sm-2.2.5.tar.gz#egg=en_core_web_sm==2.2.5 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for en_core_web_sm: filename=en_core_web_sm-2.2.5-py3-none-any.whl size=12011718 sha256=af59cf5c7a9daec2965fbfa5b48d346edccf4a9c5c2d1c85e06e649d21f74a5b
  Stored in directory: /private/var/folders/km/4_y5sdb51yg4g56t_jx0nt740000gn/T/pip-ephem-wheel-cache-r0a8h11r/wheels/77/b4/c8/395804b9a2b6864aaff3623d7b709680acc3d04f47c8162ee6
Successfully built en_core_web_sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


FileNotFoundError: file /generation/model.tar.gz not found